In [1]:
import glob
import json
import csv
import pandas as pd
import numpy as np
import itertools

In [2]:
def is_not_ascii(string):
    return string is not None and any([ord(s) >= 128 for s in string])

In [3]:
def reviews2Dataframe(path):
    review = []
    f = open(path)
    for line in itertools.islice(f,900000):
        line = line.strip()
        if not line: continue
        review.append(line)

    # Making a list of Tuples containg all the data of json files.
    reviewDataframe=[]
    for x in review:
        try:
            jdata=json.loads(x)
            reviewDataframe.append((jdata['reviewerID'],jdata['asin'],jdata['reviewerName'],jdata['overall'],jdata['reviewTime'],jdata['reviewText'],jdata['summary'])) 
        except:
            pass        

    f.close();
    dataset=pd.DataFrame(reviewDataframe,columns=['Reviewer_ID','Asin','Reviewer_Name','Overall','Review_time','Review_Text','Summary'])
    dataset['Review_time'] = dataset['Review_time'].str.slice(start=6)
    dataset['Review_time'] = dataset['Review_time'].astype(int)
    reviewDat = dataset[dataset['Review_time']>=2007]
    reviewDat.dropna()
#     reviewDat[reviewDat['Reviewer_Name'].apply(is_not_ascii)]
#     reviewDat[reviewDat['Review_Text'].apply(is_not_ascii)]
#     reviewDat[reviewDat['Summary'].apply(is_not_ascii)]
    return reviewDat


In [4]:
def meta2Dataframe(path, t):
    if t=='k':
        v = 2
    elif t=='b':
        v = 1
    review = []
    f2 = open(path)
    for line in itertools.islice(f2,500000):
        line = line.strip()
        if not line: continue
        review.append(line)

    # Making a list of Tuples containg all the data of json files.
    reviewDataframe=[]
    for x in review:
        try:
            jdata=json.loads(x)
            reviewDataframe.append((jdata['category'][v],jdata['brand'],jdata['title'],jdata['asin'], jdata['rank'])) 
        except:
            pass        

    f2.close();
    dataset2=pd.DataFrame(reviewDataframe,columns=['Category','Brand','Title','Asin', 'Rank'])
    dataset2['Rank'] = dataset2.Rank.str.replace(r'\D+', '')
    dataset2.dropna()
#     dataset2[dataset2['Category'].apply(is_not_ascii)]
#     dataset2[dataset2['Brand'].apply(is_not_ascii)]
#     dataset2[dataset2['Title'].apply(is_not_ascii)]
    return dataset2

In [5]:
##### Read review Books
reviewBooks = reviews2Dataframe("Books_5.json")
metaBooks = meta2Dataframe("meta_Books.json", 'b')

In [6]:
#### Read kindle
reviewKindle = reviews2Dataframe("Kindle_Store_5.json")
metaKindle = meta2Dataframe("meta_Kindle_Store.json", 'k')

In [7]:
mergedBooks = pd.merge(left=reviewBooks, right=metaBooks, left_on='Asin', right_on='Asin')

In [8]:
mergedKindle = pd.merge(left=reviewKindle, right=metaKindle, left_on='Asin', right_on='Asin')

In [9]:
# Find unique genres
uniqueBooksCategory = mergedBooks['Category'].unique()
uniqueKindleCategory = mergedKindle['Category'].unique()

commonCategory = np.intersect1d(uniqueBooksCategory, uniqueKindleCategory)


In [10]:
Books = mergedBooks[mergedBooks['Category'].isin(commonCategory)]
Kindle = mergedKindle[mergedKindle['Category'].isin(commonCategory)]

In [11]:
Books

,Reviewer_ID,Asin,Reviewer_Name,Overall,Review_time,Review_Text,Summary,Category,Brand,Title,Rank
0,A10000012B7CGYKOMPQ4L,000100039X,Adam,5.0,2012,Spiritually and mentally inspiring! A book tha...,Wonderful!,Literature & Fiction,Visit Amazon's Kahlil Gibran Page,The Prophet,1810945
1,A1BM81XB4QHOA3,000100039X,"Ahoro Blethends ""Seriously""",5.0,2014,This book provides a reflection that you can a...,Must Read for Life Afficianados,Literature & Fiction,Visit Amazon's Kahlil Gibran Page,The Prophet,1810945
2,A1MOSTXNIO5MPJ,000100039X,Alan Krug,5.0,2011,I first read THE PROPHET in college back in th...,Timeless for every good and bad time in your l...,Literature & Fiction,Visit Amazon's Kahlil Gibran Page,The Prophet,1810945
3,A3V1MKC2BVWY48,000100039X,Alex Dawson,5.0,2014,Reading this made my mind feel like a still po...,This book will bring you peace,Literature & Fiction,Visit Amazon's Kahlil Gibran Page,The Prophet,1810945
4,A12387207U8U24,000100039X,Alex,5.0,2008,"As you read, Gibran's poetry brings spiritual ...",Graet Work,Literature & Fiction,Visit Amazon's Kahlil Gibran Page,The Prophet,1810945
5,A29TRDMK51GKZR,000100039X,Alpine Plume,5.0,2013,"Deep, moving dramatic verses of the heart and ...",Such Beauty,Literature & Fiction,Visit Amazon's Kahlil Gibran Page,The Prophet,1810945
6,A3FI0744PG1WYG,000100039X,"Always Reading ""tkm""",5.0,2014,This is a timeless classic. Over the years I'...,The Prophet,Literature & Fiction,Visit Amazon's Kahlil Gibran Page,The Prophet,1810945
7,A2LBBQHYLEHM7P,000100039X,"Amazon Customer ""Full Frontal Nerdity""",5.0,2013,An amazing work. Realizing extensive use of Bi...,A Modern Classic,Literature & Fiction,Visit Amazon's Kahlil Gibran Page,The Prophet,1810945
8,A1340OFLZBW5NG,000100039X,Amazon Customer,5.0,2009,I LOVE this book... his writing seems to just ...,Perhaps the greatest book that I have ever read,Literature & Fiction,Visit Amazon's Kahlil Gibran Page,The Prophet,1810945
9,A2KU9IU07LOJS1,000100039X,Amazon Customer,5.0,2013,This book has been a classic for many years. I...,Great classic that everyone should read,Literature & Fiction,Visit Amazon's Kahlil Gibran Page,The Prophet,1810945


In [12]:
import sys
reload(sys)
sys.setdefaultencoding('utf8')

In [ ]:
# to csv
Books.to_csv('finalbooks.csv')
Kindle.to_csv('finalkindle.csv')

In [ ]:
# Data for grouped bar chart

In [14]:
def CountGenres(frame):
    year = [2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014]
    column_names = ['Genres', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014']
    genres = frame['Category'].unique().tolist()
    d = {}
    d['Genres'] = genres
    for y in year:
        f = frame[frame['Review_time']==y]
        vals = f['Category'].value_counts().tolist()
        d[str(y)]=vals
    df = pd.DataFrame.from_dict(d)
    df = df.reindex(columns = column_names)
    return df
    

In [15]:
## Count frequency of genres every year
booksGenreCount = CountGenres(Books)
# kindleGenreCount = CountGenres(Kindle)

In [16]:
## Read kindle
year = [2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014]
column_names = ['Genres', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014']
genres = Kindle['Category'].unique().tolist()
d = {}
d['Genres'] = genres
print d
for y in year:
    f = Kindle[Kindle['Review_time']==y]
    k = f['Category'].value_counts().keys().tolist()
    cnt = f['Category'].value_counts().tolist()
    res = []
    for i,key in enumerate(genres):
        if key in k:
            res.append(cnt[k.index(key)])
        else:
            res.append(0)
#     vals = f['Category'].value_counts().tolist()
    d[str(y)]=res
df = pd.DataFrame.from_dict(d)
kindleGenreCount = df.reindex(columns = column_names)

In [17]:
kindleGenreCount

,Genres,2007,2008,2009,2010,2011,2012,2013,2014
0,Literature & Fiction,7,52,126,750,6469,23367,96993,77390
1,Science Fiction & Fantasy,0,4,18,95,1637,5024,15216,9528
2,Romance,1,21,52,282,2558,9513,41889,31990
3,"Mystery, Thriller & Suspense",1,1,7,37,820,2522,7897,7406
4,Religion & Spirituality,0,0,3,96,364,1879,8135,6129
5,History,0,0,0,0,29,274,1120,569
6,Education & Teaching,0,0,0,0,2,270,766,355
7,Arts & Photography,0,1,0,0,23,179,518,202
8,Computers & Technology,0,2,4,1,21,217,536,262
9,"Cookbooks, Food & Wine",0,0,4,0,36,1613,4091,3277


In [18]:
booksGenreCount

,Genres,2007,2008,2009,2010,2011,2012,2013,2014
0,Literature & Fiction,7354,8237,9141,10019,12762,21256,47992,27859
1,Science & Math,1631,1955,2695,3748,4632,5988,12679,7329
2,History,1599,1859,2398,2931,3389,5446,9247,5242
3,Travel,1314,1780,2325,2308,2812,3563,7403,4263
4,Biographies & Memoirs,1219,1554,2226,2203,2369,2973,5762,4260
5,"Cookbooks, Food & Wine",1128,1474,1818,1838,1859,2378,5148,2913
6,Business & Money,729,1302,1338,1710,1786,1766,4186,1856
7,Sports & Outdoors,706,731,1057,1317,1595,1762,4009,1605
8,"Mystery, Thriller & Suspense",616,665,908,1309,1442,1678,3079,1416
9,Science Fiction & Fantasy,510,576,702,1113,1139,1531,2547,1328


In [248]:
booksGenreCount.to_csv('booksGenreCount.csv')
kindleGenreCount.to_csv('kindleGenreCount.csv')